In [10]:
def get_student_result(roll, db_path=DB_PATH):
    conn = get_conn(db_path)
    cur = conn.cursor()
    cur.execute("""
        SELECT s.roll, s.name, s.class, m.subject, m.exam_type, m.marks 
        FROM students s 
        LEFT JOIN marks m ON s.roll = m.roll 
        WHERE s.roll = ?;
    """, (str(roll),))
    
    rows = cur.fetchall()
    conn.close()
    
    if not rows:
        return None
    
    # Student basic info
    student_info = {
        "roll": rows[0][0],
        "name": rows[0][1],
        "class": rows[0][2]
    }
    
    # Marks
    marks_rows = [
        {"subject": r[3], "exam_type": r[4], "marks": r[5]}
        for r in rows if r[3] is not None
    ]
    
    total = sum(m['marks'] for m in marks_rows) if marks_rows else 0
    count = len(marks_rows)
    percentage = (total / (count * 100) * 100) if count > 0 else 0
    
    return {
        "student": student_info,
        "marks": marks_rows,
        "total": total,
        "count": count,
        "percentage": round(percentage, 2)
    }

# Test for roll 101
get_student_result("101")

{'student': {'roll': '101', 'name': 'Aman Kumar', 'class': '10A'},
 'marks': [{'subject': 'Math', 'exam_type': 'Unit Test', 'marks': 78},
  {'subject': 'Science', 'exam_type': 'Midterm', 'marks': 88}],
 'total': 166,
 'count': 2,
 'percentage': 83.0}

In [9]:

update_mark(1, 80)


delete_mark(3)

print("After update/delete:")
display(get_all_marks())

After update/delete:


,id,roll,subject,exam_type,marks,created_at
0,7,101,Math,Unit Test,78,2025-11-17 16:31:43
1,8,101,Science,Midterm,88,2025-11-17 16:31:43
2,9,102,Math,Unit Test,92,2025-11-17 16:31:43


In [8]:
print("Students:")
display(get_all_students())

print("\nMarks:")
display(get_all_marks())

Students:


,roll,name,class,created_at
0,101,Aman Kumar,10A,2025-11-17 16:31:43
1,102,Neha Singh,10A,2025-11-17 16:31:43



Marks:


,id,roll,subject,exam_type,marks,created_at
0,7,101,Math,Unit Test,78,2025-11-17 16:31:43
1,8,101,Science,Midterm,88,2025-11-17 16:31:43
2,9,102,Math,Unit Test,92,2025-11-17 16:31:43


In [7]:
conn = get_conn()
cur = conn.cursor()
cur.execute("DELETE FROM marks;")
cur.execute("DELETE FROM students;")
conn.commit()
conn.close()

add_student("101", "Aman Kumar", "10A")
add_student("102", "Neha Singh", "10A")


add_mark("101", "Math", "Unit Test", 78)
add_mark("101", "Science", "Midterm", 88)
add_mark("102", "Math", "Unit Test", 92)

print("Sample data added!")

Sample data added!


In [4]:
def get_conn(db_path=DB_PATH):
    return sqlite3.connect(db_path)

def add_student(roll, name, cls, db_path=DB_PATH):
    conn = get_conn(db_path)
    cur = conn.cursor()
    cur.execute("INSERT OR REPLACE INTO students (roll, name, class) VALUES (?, ?, ?);", 
                (str(roll), name, cls))
    conn.commit()
    conn.close()

def add_mark(roll, subject, exam_type, marks, db_path=DB_PATH):
    conn = get_conn(db_path)
    cur = conn.cursor()
    cur.execute("INSERT INTO marks (roll, subject, exam_type, marks) VALUES (?, ?, ?, ?);", 
                (str(roll), subject, exam_type, int(marks)))
    conn.commit()
    conn.close()

def get_all_students(db_path=DB_PATH):
    conn = get_conn(db_path)
    df = pd.read_sql_query("SELECT * FROM students ORDER BY roll;", conn)
    conn.close()
    return df

def get_all_marks(db_path=DB_PATH):
    conn = get_conn(db_path)
    df = pd.read_sql_query("SELECT * FROM marks ORDER BY id;", conn)
    conn.close()
    return df

def update_mark(mark_id, marks, db_path=DB_PATH):
    conn = get_conn(db_path)
    cur = conn.cursor()
    cur.execute("UPDATE marks SET marks = ? WHERE id = ?;", (int(marks), int(mark_id)))
    conn.commit()
    conn.close()

def delete_mark(mark_id, db_path=DB_PATH):
    conn = get_conn(db_path)
    cur = conn.cursor()
    cur.execute("DELETE FROM marks WHERE id = ?;", (int(mark_id),))
    conn.commit()
    conn.close()

In [3]:
def create_tables(db_path=DB_PATH):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS students (
        roll TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        class TEXT NOT NULL,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP
    );
    """)
    cur.execute("""
    CREATE TABLE IF NOT EXISTS marks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        roll TEXT NOT NULL,
        subject TEXT NOT NULL,
        exam_type TEXT NOT NULL,
        marks INTEGER NOT NULL,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (roll) REFERENCES students(roll) ON DELETE CASCADE
    );
    """)
    conn.commit()
    conn.close()

create_tables()
print("Tables created!")

Tables created!


In [2]:
import sqlite3
from datetime import datetime
import pandas as pd  # optional but helps display tables

DB_PATH = "student_db.sqlite"